In [405]:
!python -V

import pandas as pd
import numpy as np  

import seaborn as sns
import matplotlib.pyplot as plt
import json

import matplotlib.pyplot as plt

Python 3.9.12


In [406]:
import argparse
import os
import pickle

In [407]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error


import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [408]:
import mlflow
 
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("glucose-experiment")

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='glucose-experiment', tags={}>

In [409]:
def read_dataframe(filename: str):
    # load file
    data = pd.read_json(filename)
    # Rearrange cols
    cols = pd.Series(data.wn_interpolated[0]).astype(int)
    list_arrays = list(data.spectrum_interpolated)

    data = pd.DataFrame(data=list_arrays, columns=cols, index=data.glucose).reset_index()

    data.index = data.index.set_names(['experiment_number'])
    data.reset_index(inplace=True)
    
    
    data = pd.melt(data, id_vars=['glucose','experiment_number'], value_vars=data.iloc[:,2:].columns)
    data.experiment_number = data.experiment_number.apply(lambda x: x+1) 

    return data


def preprocess(data: pd.DataFrame, dv: DictVectorizer, fit_dv: bool = False):
    features = ['variable','value']
    dicts = data[features].to_dict(orient='records')
    dicts
    if fit_dv:
        X = dv.fit_transform(dicts)
    else:
        X = dv.transform(dicts)
    return X, dv

In [410]:

data_path = os.path.join("./dataset.json")
data = read_dataframe(data_path)

data_train = data[data.experiment_number <= data.experiment_number.max()*0.70]
data_val = data[(data.experiment_number > data.experiment_number.max()*0.70) & (data.experiment_number <= data.experiment_number.max()*0.85)]
data_test = data[data.experiment_number > data.experiment_number.max()*0.85]   
    
# extract the target
target = 'glucose'
y_train = data_train[target].values
y_val = data_val[target].values
y_test = data_test[target].values

# fit the dictvectorizer and preprocess data
dv = DictVectorizer()
X_train, dv = preprocess(data_train, dv, fit_dv=True)
X_val, _ = preprocess(data_val, dv, fit_dv=False)
X_test, _ = preprocess(data_test, dv, fit_dv=False)



In [411]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [412]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [414]:
def load_pickle(filename: str):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)


with mlflow.start_run():

    def run(data_path):

        X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
        X_valid, y_valid = load_pickle(os.path.join(data_path, "val.pkl"))
        
        

        rf = RandomForestRegressor(max_depth=10, random_state=0)
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_valid)

        rmse = mean_squared_error(y_valid, y_pred, squared=False)

        


if __name__ == '__main__':

    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--data_path",
        default="./output",
        help="the location where the processed NYC taxi trip data was saved."
    )
    args = parser.parse_args()

    run(args.data_path)



mlflow.log_artifact("models/randon_forest.b", artifact_path="randon_forest")
#mlflow.sklearn.log_model(rf, artifact_path="models")


usage: ipykernel_launcher.py [-h] [--data_path DATA_PATH]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9003 --control=9001 --hb=9000 --Session.signature_scheme="hmac-sha256" --Session.key=b"a7379f6d-4e61-4153-acf3-8d1dd1a13fe9" --shell=9002 --transport="tcp" --iopub=9004 --f=/Users/marchan/Library/Jupyter/runtime/kernel-1869KFOT4vBmctPy.json


SystemExit: 2

In [383]:
data = pd.read_json('./dataset.json')

cols = pd.Series(data.wn_interpolated[0]).astype(int)
list_arrays = list(data.spectrum_interpolated)

data = pd.DataFrame(data=list_arrays, columns=cols, index=data.glucose).reset_index()

data.index = data.index.set_names(['experiment_number'])
data.reset_index(inplace=True)
data


,experiment_number,glucose,950,951,952,953,954,955,956,957,...,1191,1192,1193,1194,1195,1196,1197,1198,1199,1200
0,0,180,0.005445,0.005084,0.004748,0.004435,0.004140,0.003857,0.003587,0.003337,...,0.003960,0.003960,0.003960,0.003960,0.003960,0.003960,0.003960,0.003960,0.003960,0.003960
1,1,160,0.005118,0.004855,0.004609,0.004379,0.004159,0.003944,0.003737,0.003544,...,0.006330,0.006330,0.006330,0.006330,0.006330,0.006330,0.006330,0.006330,0.006330,0.006330
2,2,90,0.001874,0.002117,0.002358,0.002590,0.002806,0.003002,0.003178,0.003340,...,0.008111,0.008111,0.008111,0.008111,0.008111,0.008111,0.008111,0.008111,0.008111,0.008111
3,3,90,0.001911,0.001861,0.001826,0.001804,0.001790,0.001774,0.001757,0.001744,...,0.003498,0.003498,0.003498,0.003498,0.003498,0.003498,0.003498,0.003498,0.003498,0.003498
4,4,110,0.003414,0.003461,0.003512,0.003563,0.003609,0.003643,0.003668,0.003689,...,0.005399,0.005399,0.005399,0.005399,0.005399,0.005399,0.005399,0.005399,0.005399,0.005399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1915,1915,185,0.001706,0.001735,0.001776,0.001832,0.001893,0.001950,0.002000,0.002049,...,0.003898,0.003898,0.003898,0.003898,0.003898,0.003898,0.003898,0.003898,0.003898,0.003898
1916,1916,180,0.006960,0.006720,0.006483,0.006248,0.006009,0.005764,0.005515,0.005271,...,0.006838,0.006838,0.006838,0.006838,0.006838,0.006838,0.006838,0.006838,0.006838,0.006838
1917,1917,155,0.000638,0.000834,0.001038,0.001246,0.001450,0.001641,0.001816,0.001981,...,0.004305,0.004305,0.004305,0.004305,0.004305,0.004305,0.004305,0.004305,0.004305,0.004305
1918,1918,195,0.001597,0.001607,0.001636,0.001684,0.001741,0.001798,0.001853,0.001911,...,0.003423,0.003423,0.003423,0.003423,0.003423,0.003423,0.003423,0.003423,0.003423,0.003423
